In [9]:
import pandas as pd


In [2]:
!pip3 install tensorflow-hub  tensorflow-text pandas numpy tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.7 MB/s eta 0:00:00


In [10]:
df = pd.read_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/train.csv")
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [11]:
df['text'].unique()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'Suicide bomber kills 15 in Saudi security site mosque - Reuters via World - Google News - Wall ... http://t.co/nF4IculOje',
       'Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [12]:
df1 = df[['text', 'target']]
df1.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
import re

def remove_noise(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, numeric characters, URLs, and non-ASCII characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)


    # Convert text to lowercase and remove leading/trailing whitespaces
    text = text.strip().lower()

    return text

In [14]:
df1['clean_text'] = df1['text'].apply(remove_noise)
df1.tail(20)

<ipython-input-14-7c4605d477a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['clean_text'] = df1['text'].apply(remove_noise)


,text,target,clean_text
7593,I just heard a really loud bang and everyone i...,0,i just heard a really loud bang and everyone i...
7594,A gas thing just exploded and I heard screams ...,1,a gas thing just exploded and i heard screams ...
7595,NWS: Flash Flood Warning Continued for Shelby ...,1,nws flash flood warning continued for shelby c...
7596,RT @LivingSafely: #NWS issues Severe #Thunders...,1,rt livingsafely nws issues severe thunderstorm...
7597,#??? #?? #??? #??? MH370: Aircraft debris foun...,1,mh aircraft debris found on la reunion is from...
7598,Father-of-three Lost Control of Car After Over...,1,fatherofthree lost control of car after overta...
7599,1.3 #Earthquake in 9Km Ssw Of Anza California ...,1,earthquake in km ssw of anza california iphone...
7600,Evacuation order lifted for town of Roosevelt:...,1,evacuation order lifted for town of roosevelt ...
7601,#breaking #LA Refugio oil spill may have been ...,1,breaking la refugio oil spill may have been co...
7602,a siren just went off and it wasn't the Forney...,1,a siren just went off and it wasnt the forney ...


In [15]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

# Now lets import BERT model and get embeding vectors for few sample statements
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
bert_preprocess_model = hub.KerasLayer(bert_preprocess) # l1

ModuleNotFoundError: No module named 'tensorflow_text'

In [14]:
# Binary class -> Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [15]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [16]:
model.fit(df1['clean_text'], df1['target'], epochs=10)

Epoch 1/10
238/238 [==============================] - 105s 365ms/step - loss: 0.6323 - accuracy: 0.6507 - precision: 0.6268 - recall: 0.4625
Epoch 2/10
238/238 [==============================] - 87s 365ms/step - loss: 0.5801 - accuracy: 0.7083 - precision: 0.6905 - recall: 0.5818
Epoch 3/10
238/238 [==============================] - 86s 361ms/step - loss: 0.5550 - accuracy: 0.7310 - precision: 0.7204 - recall: 0.6111
Epoch 4/10
238/238 [==============================] - 86s 363ms/step - loss: 0.5437 - accuracy: 0.7372 - precision: 0.7258 - recall: 0.6240
Epoch 5/10
238/238 [==============================] - 86s 362ms/step - loss: 0.5364 - accuracy: 0.7445 - precision: 0.7339 - recall: 0.6359
Epoch 6/10
238/238 [==============================] - 86s 363ms/step - loss: 0.5316 - accuracy: 0.7517 - precision: 0.7444 - recall: 0.6429
Epoch 7/10
238/238 [==============================] - 86s 362ms/step - loss: 0.5232 - accuracy: 0.7537 - precision: 0.7458 - recall: 0.6475
Epoch 8/10
238/238 

In [17]:
model.save("kaggle_twitter_text_classification")

In [18]:
model = tf.keras.models.load_model("kaggle_twitter_text_classification")

In [16]:
test_df = pd.read_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/test.csv")
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [17]:
test_df['clean_text'] = test_df['text'].apply(remove_noise)
test_df.tail(20)

,id,keyword,location,text,clean_text
3243,10796,wrecked,Deep in the heart of LibLand,'What manner of human being would parcel out a...,what manner of human being would parcel out a ...
3244,10797,wrecked,NaN,@Nathan26_RFC thought you said Saturday night ...,nathanrfc thought you said saturday night ther...
3245,10801,wrecked,"Canada,Ontario",I just wanna ease your mind and make you feel ...,i just wanna ease your mind and make you feel ...
3246,10804,wrecked,Love Reiss,@yakubOObs think he deactivated because his no...,yakuboobs think he deactivated because his not...
3247,10806,wrecked,Seattle Washington,RT CNBC '3 words from Disney CEO Bob Iger wrec...,rt cnbc words from disney ceo bob iger wrecke...
3248,10807,wrecked,Acey mountain islanddåÇTorontoåÈ,Smackdown tyme this should put me in a good mo...,smackdown tyme this should put me in a good mo...
3249,10816,wrecked,los angeles,@thrillhho jsyk I haven't stopped thinking abt...,thrillhho jsyk i havent stopped thinking abt r...
3250,10820,wrecked,"Brussels, Belgium",@stighefootball Begovic has been garbage. He g...,stighefootball begovic has been garbage he got...
3251,10828,wrecked,NaN,Wrecked today got my hattrick ????,wrecked today got my hattrick
3252,10836,NaN,NaN,#Ebola #EbolaOutbreak Ebola Virus: Birmingham ...,ebola ebolaoutbreak ebola virus birmingham ala...


In [21]:
predictions = model.predict(test_df['text'])

102/102 [==============================] - 38s 370ms/step


In [22]:
threshold = 0.5

# Convert decimal values to binary (0 or 1) using the threshold
predictions = (predictions > threshold).astype(int)
predictions

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [35]:
predictions_df = pd.DataFrame(predictions, columns=['target'])
predictions_df['id'] = test_df['id']
predictions_df.set_index('id', inplace=True)

# Resetting the index to None

predictions_df.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit.csv")

In [36]:
predictions_df

,target
id,
0,1
2,0
3,1
9,1
11,1
...,...
10861,0
10865,1
10868,1


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [30]:
df1_class_0 = df1[df1['target'] == 0]
df1_class_1 = df1[df1['target'] == 1]

# Determine the number of samples to select from each class
num_samples = min(len(df1_class_0), len(df1_class_1))

# Randomly sample an equal number of samples from each class
df1_balanced = pd.concat([df1_class_0.sample(num_samples, random_state=42), df1_class_1.sample(num_samples, random_state=42)])

# Shuffle the balanced DataFrame
df1_balanced = df1_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [25]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
    ('clf', LinearSVC())  # LinearSVC classifier
])


In [31]:
# pipeline.fit(df1_balanced['clean_text'], df1_balanced['target'])
pipeline.fit(df1_balanced['clean_text'], df1_balanced['target'])


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [32]:
predictions_svc = pipeline.predict(test_df['clean_text'])

In [33]:
predictions_svc

array([1, 1, 1, ..., 1, 1, 1])

In [34]:
predictions_svc_df = pd.DataFrame(predictions_svc, columns=['target'])
predictions_svc_df['target'].unique()
predictions_svc_df['id'] = test_df['id']
predictions_svc_df.set_index('id', inplace=True)
predictions_svc_df.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_svc_v5.csv")

In [20]:
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
    ('clf', MultinomialNB())  # Multinomial Naive Bayes classifier
])

In [22]:
# pipeline_nb.fit(df1_balanced['clean_text'], df1_balanced['target'])
pipeline_nb.fit(df1['clean_text'], df1['target'])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
predictions_nb = pipeline_nb.predict(test_df['clean_text'])

In [24]:
predictions_nb_df = pd.DataFrame(predictions_nb, columns=['target'])
predictions_nb_df['target'].unique()
predictions_nb_df['id'] = test_df['id']
predictions_nb_df.set_index('id', inplace=True)
predictions_nb_df.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_nb_v3.csv")

In [25]:
grid_search_pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
     ('feature_selection', SelectKBest(chi2)),
    ('clf', MultinomialNB())  # Multinomial Naive Bayes classifier
])

In [26]:
param_grid = {
    'tfidf__max_features': [1000, 5000, 10000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'feature_selection__k': [100, 500, 1000],
    'clf__alpha': [0.1, 0.5, 1.0],
}

In [27]:
grid_search_nb = GridSearchCV(grid_search_pipeline_nb, param_grid, cv=5, n_jobs=-1)

In [28]:
# grid_search_nb.fit(df1_balanced['clean_text'], df1_balanced['target'])
grid_search_nb.fit(df1['clean_text'], df1['target'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function chi2 at 0x7986ffc2a560>)),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1, 0.5, 1.0],
                         'feature_selection__k': [100, 500, 1000],
                         'tfidf__max_features': [1000, 5000, 10000],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]})

In [29]:
grid_search_nb_predictions = grid_search_nb.predict(test_df['text'])

In [30]:
grid_search_nb_predictions = pd.DataFrame(grid_search_nb_predictions, columns=['target'])
grid_search_nb_predictions['target'].unique()
grid_search_nb_predictions['id'] = test_df['id']
grid_search_nb_predictions.set_index('id', inplace=True)
grid_search_nb_predictions.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_grid_search_nb_predictions_v2.csv")

In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier

In [20]:
pipeline_xg = Pipeline([
    ('features', FeatureUnion([
        ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
        ('lsa', Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('svd', TruncatedSVD(n_components=50))  # TruncatedSVD for dimensionality reduction
        ]))
    ])),
    ('clf', XGBClassifier())  # XGBoost classifier
])

In [21]:
pipeline_xg.fit(df1['clean_text'], df1['target'])

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf', TfidfVectorizer()),
                                                ('lsa',
                                                 Pipeline(steps=[('tfidf',
                                                                  TfidfVectorizer()),
                                                                 ('svd',
                                                                  TruncatedSVD(n_components=50))]))])),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=No...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [22]:
predictions_xg = pipeline_xg.predict(test_df['text'])

In [23]:
predictions_xg_df = pd.DataFrame(predictions_xg, columns=['target'])
predictions_xg_df['target'].unique()
predictions_xg_df['id'] = test_df['id']
predictions_xg_df.set_index('id', inplace=True)
predictions_xg_df.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_xg_v1.csv")

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
pipeline_rf = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
    ('clf', RandomForestClassifier())  # Random Forest classifier
])

In [11]:
pipeline_rf.fit(df1['clean_text'], df1['target'])

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [12]:
predictions_rf = pipeline_rf.predict(test_df['text'])

In [13]:
predictions_rf = pd.DataFrame(predictions_rf, columns=['target'])
predictions_rf['target'].unique()
predictions_rf['id'] = test_df['id']
predictions_rf.set_index('id', inplace=True)
predictions_rf.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_rf_v1.csv")

In [17]:
from sklearn.linear_model import LogisticRegression
pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer()),  # TF-IDF vectorizer
    ('clf', LogisticRegression())  # Logistic Regression classifier
])
# pipeline_lr.fit(df1['clean_text'], df1['target'])
pipeline_lr.fit(df1_balanced['clean_text'], df1_balanced['target'])

predictions_lr = pipeline_lr.predict(test_df['text'])


In [18]:
predictions_lr = pd.DataFrame(predictions_lr, columns=['target'])
predictions_lr['target'].unique()
predictions_lr['id'] = test_df['id']
predictions_lr.set_index('id', inplace=True)
predictions_lr.to_csv("/content/drive/MyDrive/ds-ml/text-classification-bert-and-tf/submit_lr_v2.csv")